In [1]:
import numpy as np
import  functools
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

In [2]:
# 汉字字体，优先使用楷体，找不到则使用黑体
plt.rcParams['font.sans-serif'] = ['Kaitt', 'SimHei']
 
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False

In [3]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [5]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [6]:
df_data_交易_去除抹账后 = df_data_交易_去除抹账后.sort_values(["交易日期" , "交易时间"])

In [7]:
df_tmp = pd.DataFrame(data=None,columns=['账户代号','最大交易金额发生日期'])
for (name, df) in df_data_交易_去除抹账后.groupby("账户代号") :
    df_tmp = df_tmp.append({'账户代号': name , "最大交易金额发生日期" : df.nlargest(1 , "交易金额")["交易日期"].iloc[0] }, ignore_index=True)
df_tz_last_and_biggist = df_tmp.copy()

C:\Users\14714\AppData\Local\Temp\ipykernel_24388\4164645206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append({'账户代号': name , "最大交易金额发生日期" : df.nlargest(1 , "交易金额")["交易日期"].iloc[0] }, ignore_index=True)
C:\Users\14714\AppData\Local\Temp\ipykernel_24388\4164645206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append({'账户代号': name , "最大交易金额发生日期" : df.nlargest(1 , "交易金额")["交易日期"].iloc[0] }, ignore_index=True)
C:\Users\14714\AppData\Local\Temp\ipykernel_24388\4164645206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append({'账户代号': name , "最大交易金额发生日期" : df.nlargest(1 , "交易金额")["交易日期"].iloc[0] }, ignore_index=True)
C:\Users\14714\AppData\Local\Temp\ipykernel_24

In [8]:
df_tmp = df_data_交易_去除抹账后.groupby("账户代号")["交易日期"].max().reset_index()
df_tmp["交易日期"] = pd.to_datetime(df_tmp["交易日期"])
df_tmp.columns = ["账户代号" , "最后交易日期"]
df_tmp["最后交易日期"] = pd.to_datetime(df_tmp["最后交易日期"], format='%Y-%m-%d')
df_tz_last_and_biggist = df_tz_last_and_biggist.merge(df_tmp , on="账户代号" , how="left")

In [10]:
df_tz_last_and_biggist["最大交易金额发生日期"] = pd.to_datetime(df_tz_last_and_biggist['最大交易金额发生日期'], format='%Y-%m-%d')
df_tz_last_and_biggist["最后交易日期"] = pd.to_datetime(df_tz_last_and_biggist['最后交易日期'], format='%Y-%m-%d')

In [12]:
type(df_tz_last_and_biggist.iloc[1 , 1])

pandas._libs.tslibs.timestamps.Timestamp